In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['wildcam-reduced', 'wildcamfiles', 'iwildcam-2019-fgvc6']


In [3]:
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import keras
import keras.backend as K
from keras.layers import Input,Dense,Conv2D,MaxPooling2D,Dropout,Flatten,BatchNormalization,Activation
from keras.layers.merge import add
from keras.models import Model,Sequential
from sklearn.model_selection import train_test_split

In [4]:
CHANNEL_AXIS = 3
stride = 1

In [38]:
#This is a basic Residual Layer


def res_layer(x,temp,filters,pooling = False,dropout = 0.0):
    temp = Conv2D(filters,(3,3),strides = stride,padding = "same")(temp)
    temp = BatchNormalization(axis = CHANNEL_AXIS)(temp)
    temp = Activation("relu")(temp)
    temp = Conv2D(filters,(3,3),strides = stride,padding = "same")(temp)

    x = add([temp,Conv2D(filters,(3,3),strides = stride,padding = "same")(x)])
    if pooling:
        x = MaxPooling2D((2,2))(x)
    if dropout != 0.0:
        x = Dropout(dropout)(x)
    temp = BatchNormalization(axis = CHANNEL_AXIS)(x)
    temp = Activation("relu")(temp)
    return x,temp

In [39]:
inp = Input(shape = (32,32,3))
x = inp
x = Conv2D(16,(3,3),strides = stride,padding = "same")(x)
x = BatchNormalization(axis = CHANNEL_AXIS)(x)
x = Activation("relu")(x)
temp = x
#from here on stack the residual layers remember to use padding only while increasing no. of filters.
x,temp = res_layer(x,temp,32,dropout = 0.2)
x,temp = res_layer(x,temp,32,dropout = 0.3)
x,temp = res_layer(x,temp,32,dropout = 0.4,pooling = True)
x,temp = res_layer(x,temp,64,dropout = 0.2)
x,temp = res_layer(x,temp,64,dropout = 0.2,pooling = True)
x,temp = res_layer(x,temp,256,dropout = 0.4)
x = temp
x = Flatten()(x)
x = Dropout(0.4)(x)
x = Dense(256,activation = "relu")(x)
x = Dropout(0.23)(x)
x = Dense(128,activation = "relu")(x)
x = Dropout(0.3)(x)
x = Dense(64,activation = "relu")(x)
x = Dropout(0.2)(x)
x = Dense(32,activation = "relu")(x)
x = Dropout(0.2)(x)
x = Dense(14,activation = "softmax")(x)

resnet_model = Model(inp,x,name = "Resnet")
resnet_model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 32, 32, 16)   448         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 32, 32, 16)   64          conv2d_21[0][0]                  
__________________________________________________________________________________________________
activation_15 (Activation)      (None, 32, 32, 16)   0           batch_normalization_15[0][0]     
__________________________________________________________________________________________________
conv2d_22 

[https://www.kaggle.com/xhlulu/reducing-image-sizes-to-32x32](http://) reduced dataset from this kernel

In [9]:


X_train = np.load("../input/wildcam-reduced/X_train.npy")
Y_train = np.load("../input/wildcam-reduced/y_train.npy")
X_test = np.load("../input/wildcam-reduced/X_test.npy")

In [10]:
X_train = X_train.astype("float32")/255.0
X_test = X_test.astype("float32")/255.0

In [ ]:
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2,patience=5, min_lr=1e-5)
callbacks = [reduce_lr]


In [ ]:
resnet_model.compile(optimizer="adam",loss = "categorical_crossentropy",metrics = ["accuracy"])

In [ ]:
resnet_model.fit(X_train,Y_train,batch_size=200,epochs = 15,validation_split=0.18,callbacks = callbacks)

In [ ]:
resnet_model.fit(X_train,Y_train,batch_size=350,epochs = 10,validation_split=0.5,callbacks = callbacks)

In [14]:
os.listdir("../input")

['wildcam-reduced', 'wildcamfiles', 'iwildcam-2019-fgvc6']

In [16]:
pred = resnet_model.predict(X_test)

In [18]:
submission_df = pd.read_csv("../input/iwildcam-2019-fgvc6/sample_submission.csv")
submission_df.head()

,Id,Predicted
0,b005e5b2-2c0b-11e9-bcad-06f10d5896c4,0
1,f2347cfe-2c11-11e9-bcad-06f10d5896c4,0
2,27cf8d26-2c0e-11e9-bcad-06f10d5896c4,0
3,f82f52c7-2c1d-11e9-bcad-06f10d5896c4,0
4,e133f50d-2c1c-11e9-bcad-06f10d5896c4,0


In [22]:
train_df = pd.read_csv("../input/iwildcam-2019-fgvc6/train.csv")
train_df.describe()

,category_id,frame_num,location,seq_num_frames,width,height
count,196299.000000,196299.000000,196299.00000,196299.000000,196299.0,196299.000000
mean,4.085069,1.404047,65.83003,1.793494,1024.0,747.486600
std,6.749477,0.739637,34.35272,1.026838,0.0,3.128948
min,0.000000,1.000000,0.00000,1.000000,1024.0,747.000000
25%,0.000000,1.000000,30.00000,1.000000,1024.0,747.000000
50%,0.000000,1.000000,70.00000,1.000000,1024.0,747.000000
75%,8.000000,2.000000,96.00000,3.000000,1024.0,747.000000
max,22.000000,5.000000,138.00000,5.000000,1024.0,768.000000


In [23]:
d = {i:0 for i in range(23)}

In [25]:
for c in train_df["category_id"]:
    d[c]+=1
d

{0: 131457,
 1: 6102,
 2: 0,
 3: 3398,
 4: 2210,
 5: 0,
 6: 0,
 7: 0,
 8: 6938,
 9: 0,
 10: 1093,
 11: 7209,
 12: 0,
 13: 8623,
 14: 1361,
 15: 0,
 16: 5975,
 17: 4759,
 18: 3035,
 19: 14106,
 20: 0,
 21: 0,
 22: 33}

In [27]:
matched_dict = {i:0 for i in range(14)}
k = 0
for i in range(23):
    if d[i] > 0:
        matched_dict[k] = i
        k+=1
matched_dict

{0: 0,
 1: 1,
 2: 3,
 3: 4,
 4: 8,
 5: 10,
 6: 11,
 7: 13,
 8: 14,
 9: 16,
 10: 17,
 11: 18,
 12: 19,
 13: 22}

In [29]:
submittable = pred.argmax(axis = 1)
print(submittable.shape)

(153730,)


In [30]:
for i in range(submittable.shape[0]):
    submittable[i] = matched_dict[submittable[i]]

In [31]:
submission_df["Predicted"] = submittable
submission_df.head()

,Id,Predicted
0,b005e5b2-2c0b-11e9-bcad-06f10d5896c4,0
1,f2347cfe-2c11-11e9-bcad-06f10d5896c4,0
2,27cf8d26-2c0e-11e9-bcad-06f10d5896c4,0
3,f82f52c7-2c1d-11e9-bcad-06f10d5896c4,0
4,e133f50d-2c1c-11e9-bcad-06f10d5896c4,0


In [32]:
submission_df.describe()

,Predicted
count,153730.000000
mean,3.064138
std,5.566472
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,19.000000


In [35]:
est = {i:0 for i in range(23)}
for i in submittable:
    est[i]+=1
est

{0: 100137,
 1: 16246,
 2: 0,
 3: 1582,
 4: 735,
 5: 0,
 6: 0,
 7: 0,
 8: 4753,
 9: 0,
 10: 4830,
 11: 10184,
 12: 0,
 13: 4792,
 14: 0,
 15: 0,
 16: 2606,
 17: 555,
 18: 3547,
 19: 3763,
 20: 0,
 21: 0,
 22: 0}

In [ ]:
resnet_model.save("resnet.h5")

In [ ]:
resnet_model.save_weights("resnet_weights.h5")

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
datagen.fit(X_train)
X_batch = datagen.flow(X_train, Y_train, batch_size=1000)

In [ ]:
history = resnet_model.history
history.history.keys()

In [ ]:
plt.plot(history.epoch,history.history["val_acc"])

In [36]:
submission_df.to_csv('submission_resnet2.csv',index=False)
